In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
df = pd.read_json('../../data/train/trainText-75-25-bow.json', lines=True)

In [2]:
test_df_bow = pd.read_json('../../data/test/testText-75-25-bow.json', lines=True)
X_test = list(test_df_bow['recipe'])
y_test = list(test_df_bow['cuisine'])

In [3]:
X_test = list(test_df_bow['recipe'])
y_test = list(test_df_bow['cuisine'])

In [4]:
df.head()

,cuisine,id,ingredients,label,numeric_recipe,recipe
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",0,romaine lettuce black olives grape tomatoes ga...,romaine lettuce black olives grape tomatoes ga...
1,greek,34471,"[ground pork, finely chopped fresh parsley, on...",0,ground pork finely chopped fresh parsley onion...,ground pork finely chopped fresh parsley onion...
2,greek,4635,"[minced garlic, dried oregano, red wine vinega...",0,minced garlic dried oregano red wine vinegar o...,minced garlic dried oregano red wine vinegar o...
3,greek,5980,"[orange, anise, cinnamon sticks, unflavored ge...",0,orange anise cinnamon sticks unflavored gelati...,orange anise cinnamon sticks unflavored gelati...
4,greek,18031,"[fresh dill, yoghurt, salt, myzithra, large eg...",0,fresh dill yoghurt salt myzithra large eggs ch...,fresh dill yoghurt salt myzithra large eggs ch...


In [5]:
recipes = list(df['recipe'])
labels = list(df['cuisine'])

In [6]:
file = open("stopwords.txt", "r")
stopwords = file.read().lower().split('\n')

In [11]:
vectorizer = CountVectorizer(ngram_range=(1,2), min_df=0.0005, stop_words=stopwords, binary=True)
vectorizer.fit(recipes)
print('Number of Features: %d'%len(vectorizer.get_feature_names()))

Number of Features: 5832


In [12]:
vectorizer.get_feature_names()

['active',
 'active dry',
 'added',
 'adobo',
 'adobo sauce',
 'agave',
 'agave nectar',
 'aged',
 'ale',
 'alfredo',
 'alfredo sauce',
 'allspice',
 'allspice berries',
 'allspice cinnamon',
 'almond',
 'almond butter',
 'almond extract',
 'almond flour',
 'almond meal',
 'almond milk',
 'almond paste',
 'almonds',
 'almonds garlic',
 'almonds salt',
 'amaretto',
 'amchur',
 'american',
 'american cheese',
 'aminos',
 'anaheim',
 'anaheim chile',
 'ancho',
 'ancho chile',
 'ancho powder',
 'anchovies',
 'anchovy',
 'anchovy fillets',
 'anchovy paste',
 'and',
 'and black',
 'and carrots',
 'and cut',
 'and drain',
 'and drained',
 'and green',
 'and pepper',
 'and sour',
 'andouille',
 'andouille sausage',
 'angel',
 'angel hair',
 'anise',
 'anise cinnamon',
 'anise clove',
 'anise seed',
 'apple',
 'apple cider',
 'apple juice',
 'apples',
 'applesauce',
 'apricot',
 'apricot jam',
 'apricot preserves',
 'apricots',
 'arbol',
 'arborio',
 'arborio rice',
 'arrowroot',
 'arrowroot po

In [8]:
#import pickle
#filename = 'vectorizer.sav'
#pickle.dump(vectorizer, open(filename,'wb'))

In [13]:
X_train = vectorizer.transform(recipes).toarray()
y_train = labels

In [14]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
from sklearn.pipeline import Pipeline
LogReg_model = Pipeline([('vectorizer', vectorizer),('LR', clf)])

In [16]:
from sklearn import metrics
y_pred = LogReg_model.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   brazilian       0.77      0.48      0.59       117
     british       0.57      0.27      0.37       201
cajun_creole       0.82      0.62      0.71       387
     chinese       0.85      0.95      0.90      3294
    filipino       0.77      0.41      0.54       189
      french       0.67      0.47      0.55      2205
       greek       0.91      0.81      0.86      1544
      indian       0.93      0.95      0.94      2119
       irish       0.65      0.42      0.51       167
     italian       0.72      0.99      0.83      4310
    jamaican       0.84      0.59      0.69       132
    japanese       0.89      0.82      0.85      1557
      korean       0.83      0.67      0.74       208
     mexican       0.91      0.95      0.93      3600
    moroccan       0.89      0.61      0.72       206
     russian       0.65      0.30      0.41       123
 southern_us       0.77      0.70      0.74      1080
     spanish       0.85    

In [17]:
input_recipe = input("Input Recipe: \n> ")
input_recipe = input_recipe.lower()
X_input = vectorizer.transform([input_recipe]).toarray()
prob = clf.predict_proba(X_input)[0]
classes = clf.classes_

print('\nIdentified Ingredients: \n> %s'%vectorizer.inverse_transform(X_input))

print('\nCuisine Probabilities:')
for i in range(len(prob)):
    print('> %s: %s%%'%(classes[i], int(prob[i]*100)))

Input Recipe: 
> Prepare and cook ingredients as below.  – In a bowl, combine the meat with the marinade. Cover the bowl and leave it in the fridge while you are working on other the ingredients. When ready, Add some cooking oil into a wok and cook the meat on medium high to high heat. It takes about 3 to 5 mins to thoroughly cook it. – Prepare the Korean cucumber side dish, fernbrake side dish and bellflower root side dish per the linked recipes. These can be prepared on a different day to save your bibimbap making time. – Separately sauté carrots, zucchini and mushrooms in a frying pan. Season them lightly with fine sea salt. – Mix the bibimbap sauce ingredients in a small bowl. Set aside. – Pan fry the eggs per your preference. (Sunny side up is a popular choice.)  Preheat an oven for 10 minutes at 220 C / 428 F. Once heated, place your stone bowl into the oven. Leave it there for 10 minutes. 10 minutes later, take it out. Drop in some sesame oil (about 1 Tbsp per medium size bowl) 

In [119]:
import pickle
filename = 'LogReg_v5.sav'
pickle.dump(LogReg_model, open(filename,'wb'))